# Here's Waldo

##### Much of this code was inspired by a [notebook](https://ui.neptune.ai/anton-morgunov/tf-test/n/model-for-inference-36c9b0c4-8d20-4d5a-aa54-5240cc8ce764/6f67c0e3-283c-45de-ae56-405aecd736c0) by Anton Morgunov.
##### Additional credit goes to the authors of the [Tensorflow Object Detection API Tutorial](https://tensorflow-object-detection-api-tutorial.readthedocs.io/)

In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm

In [2]:
import sys
path2scripts = 'models/research/'
sys.path.insert(0, path2scripts)

Before training the model you need to have labelled image data. I used [labelimg](https://github.com/tzutalin/labelImg), but there are others available. As you label your images and save them an .xml file is created along with each image.

Once all images are labelled you can split them up into train, test, and validation folders along with their .xml files. Then run the generate_tfrecord.py script found in the workspace directory of this github repo. 

(Code credit: Tensorflow Object Detection API Tutorial)

In [9]:
%%bash
cd workspace
python generate_tfrecord.py -x ./data/train -l ./data/annotations/label_map.pbtxt -o ./data/annotations/train.record

Successfully created the TFRecord file: ./data/annotations/train.record


In [10]:
%%bash
cd workspace
python generate_tfrecord.py -x ./data/test -l ./data/annotations/label_map.pbtxt -o ./data/annotations/test.record

Successfully created the TFRecord file: ./data/annotations/test.record


In [11]:
%%bash
cd workspace
python generate_tfrecord.py -x ./data/validate -l ./data/annotations/label_map.pbtxt -o ./data/annotations/validate.record

Successfully created the TFRecord file: ./data/annotations/validate.record


### Training a model

Once you have your .tfrecord files for each of your train, test, and validate images, then you are ready to start training a model.

You will need to choose a model to train from the [Tensorflow 2 Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md) and download that into a directory. 

In [26]:
%%bash
cd workspace/pre_trained_models
wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz

--2021-03-27 22:13:25--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 2607:f8b0:4007:804::2010, 142.250.72.240
Connecting to download.tensorflow.org (download.tensorflow.org)|2607:f8b0:4007:804::2010|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30736482 (29M) [application/x-tar]
Saving to: ‘efficientdet_d0_coco17_tpu-32.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 1.61M 18s
    50K .......... .......... .......... .......... ..........  0% 2.01M 16s
   100K .......... .......... .......... .......... ..........  0% 5.11M 13s
   150K .......... .......... .......... .......... ..........  0% 6.94M 11s
   200K .......... .......... .......... .......... ..........  0% 5.31M 10s
   250K .......... .......... .......... .......... ..........  0% 10.0M 8s
   300K .......... .......... .......... .......... .....

In [27]:
%%bash 
cd workspace/pre_trained_models
tar -xf efficientdet_d0_coco17_tpu-32.tar.gz
rm efficientdet_d0_coco17_tpu-32.tar.gz

**Next we will need to copy the pipeline file from within the folder we just downloaded into a new directory that we will set up to train our model**

In [28]:
%%bash
cp workspace/pre_trained_models/efficientdet_d0_coco17_tpu-32/pipeline.config \
workspace/models/efficientdet_d0_v1/pipeline.config

In [53]:
%cd workspace/models/efficientdet_d0_v1

/Users/scotthines/RPT/assignments/here_is_waldo/tensorflow/workspace/models/efficientdet_d0_v1


In [54]:
%%writefile pipeline.config
# %load pipeline.config
model {
  ssd {
    num_classes: 1 #changing this to number of classes being trained on (1 in my case: waldo)
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 512
        max_dimension: 512
        pad_to_max_dimension: true
      }
    }
    feature_extractor {
      type: "ssd_efficientnet-b0_bifpn_keras"
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.029999999329447746
          }
        }
        activation: SWISH
        batch_norm {
          decay: 0.9900000095367432
          scale: true
          epsilon: 0.0010000000474974513
        }
        force_use_bias: true
      }
      bifpn {
        min_level: 3
        max_level: 7
        num_iterations: 3
        num_filters: 64
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 1.0
        x_scale: 1.0
        height_scale: 1.0
        width_scale: 1.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
        use_matmul_gather: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    box_predictor {
      weight_shared_convolutional_box_predictor {
        conv_hyperparams {
          regularizer {
            l2_regularizer {
              weight: 3.9999998989515007e-05
            }
          }
          initializer {
            random_normal_initializer {
              mean: 0.0
              stddev: 0.009999999776482582
            }
          }
          activation: SWISH
          batch_norm {
            decay: 0.9900000095367432
            scale: true
            epsilon: 0.0010000000474974513
          }
          force_use_bias: true
        }
        depth: 64
        num_layers_before_predictor: 3
        kernel_size: 3
        class_prediction_bias_init: -4.599999904632568
        use_depthwise: true
      }
    }
    anchor_generator {
      multiscale_anchor_generator {
        min_level: 3
        max_level: 7
        anchor_scale: 4.0
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        scales_per_octave: 3
      }
    }
    post_processing {
      batch_non_max_suppression {
        score_threshold: 9.99999993922529e-09
        iou_threshold: 0.5
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SIGMOID
    }
    normalize_loss_by_num_matches: true
    loss {
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      classification_loss {
        weighted_sigmoid_focal {
          gamma: 1.5
          alpha: 0.25
        }
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    encode_background_as_zeros: true
    normalize_loc_loss_by_codesize: true
    inplace_batchnorm_update: true
    freeze_batchnorm: false
    add_background_class: false
  }
}
train_config {
  batch_size: 8 #change this to 8 to account for lower computing capacity
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    random_scale_crop_and_pad_to_square {
      output_size: 512
      scale_min: 0.10000000149011612
      scale_max: 2.0
    }
  }
  sync_replicas: true
  optimizer {
    momentum_optimizer {
      learning_rate {
        cosine_decay_learning_rate {
          learning_rate_base: 0.07999999821186066
          total_steps: 300000
          warmup_learning_rate: 0.0010000000474974513
          warmup_steps: 2500
        }
      }
      momentum_optimizer_value: 0.8999999761581421
    }
    use_moving_average: false
  }
  fine_tune_checkpoint: "./pre_trained_models/efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0" #this gets changed to the checkpoint of original model
  num_steps: 800
  startup_delay_steps: 0.0
  replicas_to_aggregate: 8
  max_number_of_boxes: 100
  unpad_groundtruth_tensors: false
  fine_tune_checkpoint_type: "detection" #this gets changed to detection for our purposes
  use_bfloat16: false # this is changed to false
  fine_tune_checkpoint_version: V2
}
train_input_reader: {
  label_map_path: "./data/annotations/label_map.pbtxt" #this changes to path of label_map
  tf_record_input_reader {
    input_path: "./data/annotations/train.record" #this changes to path of train tfrecord file
  }
}

eval_config: {
  metrics_set: "coco_detection_metrics"
  use_moving_averages: false
  batch_size: 1;
}

eval_input_reader: {
  label_map_path: "data/annotations/label_map.pbtxt" #this changes to path of label_map
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "data/annotations/validate.record" #this changes to path of validation tfrecord file
  }
}



Writing pipeline.config


Now we have everything that we need to run our `model_main_tf2.py` file. Which for me is located in the workspace directory.

In [55]:
%cd ../../..

/Users/scotthines/RPT/assignments/here_is_waldo/tensorflow


In [ ]:
%%bash
cd workspace
python model_main_tf2.py \
  --pipeline_config_path=./models/efficientdet_d0_v1/pipeline.config \
  --model_dir=./models/efficientdet_d0_v1 \
  --checkpoint_every_n=25 \
  --num_workers=4 \
  --alsologtostderr
;

While the cell above is running you can monitor the progress of your models training by opening up a terminal window and running the tensorboard command. The format looks something like this.

`tensorboard --logdir <dir of train folder created in the same folder as your edited pipeline>`

for me this looks like this from my workspace directory.

`tensorboard --logdir ./models/efficientdet_d0_v1/train`

Once you get that up and running then you can navigate to http://localhost:6006/ in your browser and see the tensorboard.